# ADS 509 Module 1: APIs and Web Scraping

This notebook has two parts. In the first part, you will scrape lyrics from AZLyrics.com. In the second part, you'll run code that verifies the completeness of your data pull. 

For this assignment you have chosen two musical artists who have at least 20 songs with lyrics on AZLyrics.com. We start with pulling some information and analyzing them.


## General Assignment Instructions

These instructions are included in every assignment, to remind you of the coding standards for the class. Feel free to delete this cell after reading it. 

One sign of mature code is conforming to a style guide. We recommend the [Google Python Style Guide](https://google.github.io/styleguide/pyguide.html). If you use a different style guide, please include a cell with a link. 

Your code should be relatively easy-to-read, sensibly commented, and clean. Writing code is a messy process, so please be sure to edit your final submission. Remove any cells that are not needed or parts of cells that contain unnecessary code. Remove inessential `import` statements and make sure that all such statements are moved into the designated cell. 

Make use of non-code cells for written commentary. These cells should be grammatical and clearly written. In some of these cells you will have questions to answer. The questions will be marked by a "Q:" and will have a corresponding "A:" spot for you. *Make sure to answer every question marked with a `Q:` for full credit.* 


# Importing Libraries

In [2]:
import os
import datetime
import re

# for the lyrics scrape section
import requests
import time
from bs4 import BeautifulSoup
from collections import defaultdict, Counter
import random

In [17]:
# Use this cell for any import statements you add
import shutil

---

# Lyrics Scrape

This section asks you to pull data by scraping www.AZLyrics.com. In the notebooks where you do that work you are asked to store the data in specific ways. 

In [59]:
artists = {'taylorswift':"https://www.azlyrics.com/t/taylorswift.html",
           'avicii':"https://www.azlyrics.com/a/avicii.html"} 
# we'll use this dictionary to hold both the artist name and the link on AZlyrics

artists

{'taylorswift': 'https://www.azlyrics.com/t/taylorswift.html',
 'avicii': 'https://www.azlyrics.com/a/avicii.html'}

## A Note on Rate Limiting

The lyrics site, www.azlyrics.com, does not have an explicit maximum on number of requests in any one time, but in our testing it appears that too many requests in too short a time will cause the site to stop returning lyrics pages. (Entertainingly, the page that gets returned seems to only have the song title to [a Tom Jones song](https://www.azlyrics.com/lyrics/tomjones/itsnotunusual.html).) 

Whenever you call `requests.get` to retrieve a page, put a `time.sleep(5 + 10*random.random())` on the next line. This will help you not to get blocked. If you _do_ get blocked, which you can identify if the returned pages are not correct, just request a lyrics page through your browser. You'll be asked to perform a CAPTCHA and then your requests should start working again. 

## Part 1: Finding Links to Songs Lyrics

That general artist page has a list of all songs for that artist with links to the individual song pages. 

Q: Take a look at the `robots.txt` page on www.azlyrics.com. (You can read more about these pages [here](https://developers.google.com/search/docs/advanced/robots/intro).) Is the scraping we are about to do allowed or disallowed by this page? How do you know? 

A: <!-- Delete this comment and put your answer here. --> 


In [60]:
# Let's set up a dictionary of lists to hold our links
lyrics_pages = defaultdict(list)
lyrics_pages

defaultdict(list, {})

In [61]:
for artist, artist_page in artists.items() :
    print(artist, artist_page)

taylorswift https://www.azlyrics.com/t/taylorswift.html
avicii https://www.azlyrics.com/a/avicii.html


In [62]:
for artist, artist_page in artists.items() :
    # request the page and sleep
    r = requests.get(artist_page)
    time.sleep(5 + 10*random.random())

    # now extract the links to lyrics pages from this page
    # store the links `lyrics_pages` where the key is the artist and the
    # value is a list of links. 

    # parse the page using beautiful soup
    soup = BeautifulSoup(r.text, 'html.parser')

    # for all the <a> tags where links contain /lyrics/
    # found this method in stack overflow here: https://stackoverflow.com/questions/3075550/how-can-i-get-href-links-from-html-using-python
    for link in soup.find_all('a', href=True):

        # get the href
        href = link.get('href')

        # check if its a lyrics link
        if '/lyrics/' in href:

            # set href as the full link
            href = 'https://www.azlyrics.com' + href
            lyrics_pages[artist].append(href)

lyrics_pages

defaultdict(list,
            {'taylorswift': ['https://www.azlyrics.com/lyrics/taylorswift/samegirl.html',
              'https://www.azlyrics.com/lyrics/taylorswift/babyblue.html',
              'https://www.azlyrics.com/lyrics/taylorswift/neverfade.html',
              'https://www.azlyrics.com/lyrics/taylorswift/fire.html',
              'https://www.azlyrics.com/lyrics/taylorswift/maryjo.html',
              'https://www.azlyrics.com/lyrics/taylorswift/myturntobeme.html',
              'https://www.azlyrics.com/lyrics/taylorswift/kidinthecrowd.html',
              'https://www.azlyrics.com/lyrics/taylorswift/iusedtofly.html',
              'https://www.azlyrics.com/lyrics/taylorswift/luckyyou.html',
              'https://www.azlyrics.com/lyrics/taylorswift/pointofview.html',
              'https://www.azlyrics.com/lyrics/taylorswift/whydoyoutellme.html',
              'https://www.azlyrics.com/lyrics/taylorswift/houstonrodeo.html',
              'https://www.azlyrics.com/lyrics/t

In [63]:
for artist, links in lyrics_pages.items():
    print(f"{artist}: {len(links)} songs")

taylorswift: 466 songs
avicii: 68 songs


Let's make sure we have enough lyrics pages to scrape. 

In [64]:
for artist, lp in lyrics_pages.items() :
    assert(len(set(lp)) > 20) 

In [65]:
# Let's see how long it's going to take to pull these lyrics 
# if we're waiting `5 + 10*random.random()` seconds 
for artist, links in lyrics_pages.items() : 
    print(f"For {artist} we have {len(links)}.")
    print(f"The full pull will take for this artist will take {round(len(links)*10/3600,2)} hours.")

For taylorswift we have 466.
The full pull will take for this artist will take 1.29 hours.
For avicii we have 68.
The full pull will take for this artist will take 0.19 hours.


## Part 2: Pulling Lyrics

Now that we have the links to our lyrics pages, let's go scrape them! Here are the steps for this part. 

1. Create an empty folder in our repo called "lyrics". 
1. Iterate over the artists in `lyrics_pages`. 
1. Create a subfolder in lyrics with the artist's name. For instance, if the artist was Cher you'd have `lyrics/cher/` in your repo.
1. Iterate over the pages. 
1. Request the page and extract the lyrics from the returned HTML file using BeautifulSoup.
1. Use the function below, `generate_filename_from_url`, to create a filename based on the lyrics page, then write the lyrics to a text file with that name. 


In [66]:
def generate_filename_from_link(link) :
    
    if not link :
        return None
    
    # drop the http or https and the html
    name = link.replace("https","").replace("http","")
    name = link.replace(".html","")

    name = name.replace("/lyrics/","")
    
    # replace useless chareacters with UNDERSCORE
    name = name.replace("://","").replace(".","_").replace("/","_")
    
    # tack on .txt
    name = name + ".txt"
    
    return(name)


In [71]:
# Make the lyrics folder here. If you'd like to practice your programming, add functionality 
# that checks to see if the folder exists. If it does, then use shutil.rmtree to remove it and create a new one.

if os.path.isdir("lyrics") : 
    shutil.rmtree("lyrics/")

os.mkdir("lyrics")

In [68]:
for artist, links in lyrics_pages.items():
    print(artist, links)

taylorswift ['https://www.azlyrics.com/lyrics/taylorswift/samegirl.html', 'https://www.azlyrics.com/lyrics/taylorswift/babyblue.html', 'https://www.azlyrics.com/lyrics/taylorswift/neverfade.html', 'https://www.azlyrics.com/lyrics/taylorswift/fire.html', 'https://www.azlyrics.com/lyrics/taylorswift/maryjo.html', 'https://www.azlyrics.com/lyrics/taylorswift/myturntobeme.html', 'https://www.azlyrics.com/lyrics/taylorswift/kidinthecrowd.html', 'https://www.azlyrics.com/lyrics/taylorswift/iusedtofly.html', 'https://www.azlyrics.com/lyrics/taylorswift/luckyyou.html', 'https://www.azlyrics.com/lyrics/taylorswift/pointofview.html', 'https://www.azlyrics.com/lyrics/taylorswift/whydoyoutellme.html', 'https://www.azlyrics.com/lyrics/taylorswift/houstonrodeo.html', 'https://www.azlyrics.com/lyrics/taylorswift/alittlemorelikeyou.html', 'https://www.azlyrics.com/lyrics/taylorswift/onesidedgoodbye.html', 'https://www.azlyrics.com/lyrics/taylorswift/rideon.html', 'https://www.azlyrics.com/lyrics/taylo

In [72]:
url_stub = "https://www.azlyrics.com" 
start = time.time()

total_pages = 0 

# needed to update this part to get both artist and each link or else it would only iterate through one artists links for both subfolders
for artist, links in lyrics_pages.items(): 

    # Use this space to carry out the following steps: 
    
    # 1. Build a subfolder for the artist

    # create folder for the artist if not already exist
    artist_path = os.path.join("lyrics", artist)
    os.makedirs(artist_path, exist_ok=True)

    # 2. Iterate over the lyrics pages

    # iterate over at most 20
    for link in links[:20]:

    # 3. Request the lyrics page. 

        # make request
        r = requests.get(link)
        
        # Don't forget to add a line like `time.sleep(5 + 10*random.random())`
        # to sleep after making the request

        # sleep timer after request
        time.sleep(5 + 10*random.random())

    # 4. Extract the title and lyrics from the page.

        # parse html
        soup = BeautifulSoup(r.text, 'html.parser')

        # get the title, found this method from github here: https://github.com/a-montanari/webscraping_lyrics/blob/main/scrape_functions.py#L40
        title = soup.find_all("b")[1].get_text().replace('"', '')

        # get the lyrics, found this method from stack overflow here: https://stackoverflow.com/questions/67992715/how-do-i-scrape-lyrics-from-a-list-of-song-lyric-urls
        lyrics = soup.select_one(".ringtone ~ div").get_text(strip=True, separator="\n")

        # get rid of all the "\n"
        cleaned_lyrics = lyrics.replace('\n', ' ')  # Replace with a space to maintain separation of words and sections

        # set title, next line and then lyrics as content
        content = title + '\n\n' + cleaned_lyrics
        
        # create filename from URL using function above
        filename = generate_filename_from_link(link)

        # set filepath as the subfolder plus file name
        filepath = os.path.join(artist_path, filename)

    # 5. Write out the title, two returns ('\n'), and the lyrics. Use `generate_filename_from_url`
    #    to generate the filename. 
    
        # write the content as text file to the file path
        with open(filepath, 'w', encoding='utf-8') as file:
            file.write(content)

    # Remember to pull at least 20 songs per artist. It may be fun to pull all the songs for the artist
    

In [73]:
print(f"Total run time was {round((time.time() - start)/3600,2)} hours.")

Total run time was 0.12 hours.


---

# Evaluation

This assignment asks you to pull data by scraping www.AZLyrics.com.  After you have finished the above sections , run all the cells in this notebook. Print this to PDF and submit it, per the instructions.

In [74]:
# Simple word extractor from Peter Norvig: https://norvig.com/spell-correct.html
def words(text): 
    return re.findall(r'\w+', text.lower())

## Checking Lyrics 

The output from your lyrics scrape should be stored in files located in this path from the directory:
`/lyrics/[Artist Name]/[filename from URL]`. This code summarizes the information at a high level to help the instructor evaluate your work. 

In [75]:
artist_folders = os.listdir("lyrics/")
artist_folders = [f for f in artist_folders if os.path.isdir("lyrics/" + f)]

for artist in artist_folders : 
    artist_files = os.listdir("lyrics/" + artist)
    artist_files = [f for f in artist_files if 'txt' in f or 'csv' in f or 'tsv' in f]

    print(f"For {artist} we have {len(artist_files)} files.")

    artist_words = []

    for f_name in artist_files : 
        with open("lyrics/" + artist + "/" + f_name) as infile : 
            artist_words.extend(words(infile.read()))

            
    print(f"For {artist} we have roughly {len(artist_words)} words, {len(set(artist_words))} are unique.")


For avicii we have 20 files.
For avicii we have roughly 4871 words, 638 are unique.
For taylorswift we have 20 files.
For taylorswift we have roughly 5724 words, 834 are unique.


### References:

- getting the href from each artists page: https://stackoverflow.com/questions/3075550/how-can-i-get-href-links-from-html-using-python

- getting the title from the lyrics page: https://github.com/a-montanari/webscraping_lyrics/blob/main/scrape_functions.py#L40

- getting all the lyrics from the lyrics page: https://stackoverflow.com/questions/67992715/how-do-i-scrape-lyrics-from-a-list-of-song-lyric-urls